In [ ]:
# Mount Google Drive to access files (if needed)
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Dropout, Input, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Check if GPU is available and use it
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 0:
    print("No GPU available. Using CPU instead.")
else:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f'GPU {physical_devices[0]} available: True')

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define a function for text preprocessing (stemming and cleaning)
def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

# Load dataset (adjust path as per your file location)
news_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/fakeNewsData.csv')
news_dataset = news_dataset.fillna('')

# Combine author and title into content
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']
news_dataset['content'] = news_dataset['content'].apply(preprocess_text)

# Split dataset into train and test sets
X = news_dataset['content']
y = news_dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorization using CountVectorizer
cv = CountVectorizer(max_features=5000)
X_train_cv = cv.fit_transform(X_train).toarray()
X_test_cv = cv.transform(X_test).toarray()

# Define CNN with Attention
input_layer = Input(shape=(X_train_cv.shape[1],))
dense_layer = Dense(128, activation='relu')(input_layer)
dense_layer = Dense(64, activation='relu')(dense_layer)

conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(tf.expand_dims(dense_layer, -1))
attention_data = Conv1D(filters=128, kernel_size=3, activation='relu')(tf.expand_dims(dense_layer, -1))
attention_layer = Attention()([conv_layer, attention_data])
flatten_layer = GlobalMaxPooling1D()(attention_layer)
dropout_layer = Dropout(0.5)(flatten_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model_attention = Model(inputs=input_layer, outputs=output_layer)

model_attention.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

print("\nTraining CNN with Attention and CountVectorizer Features...")
history_attention = model_attention.fit(X_train_cv, y_train, epochs=20, batch_size=32, validation_data=(X_test_cv, y_test), verbose=1)

Mounted at /content/drive
No GPU available. Using CPU instead.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.



Training CNN with Attention and CountVectorizer Features...
Epoch 1/20
520/520 [==============================] - 16s 29ms/step - loss: 0.3474 - accuracy: 0.9016 - val_loss: 0.1550 - val_accuracy: 0.9834
Epoch 2/20
520/520 [==============================] - 16s 30ms/step - loss: 0.0779 - accuracy: 0.9950 - val_loss: 0.0687 - val_accuracy: 0.9863
Epoch 3/20
520/520 [==============================] - 15s 28ms/step - loss: 0.0300 - accuracy: 0.9985 - val_loss: 0.0604 - val_accuracy: 0.9856
Epoch 4/20
520/520 [==============================] - 14s 28ms/step - loss: 0.0161 - accuracy: 0.9994 - val_loss: 0.0515 - val_accuracy: 0.9882
Epoch 5/20
520/520 [==============================] - 14s 28ms/step - loss: 0.0108 - accuracy: 0.9994 - val_loss: 0.0567 - val_accuracy: 0.9873
Epoch 6/20
520/520 [==============================] - 14s 28ms/step - loss: 0.0090 - accuracy: 0.9996 - val_loss: 0.0585 - val_accuracy: 0.9877
Epoch 7/20
520/520 [==============================] - 14s 28ms/step - loss:

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, matthews_corrcoef

# Accuracy
loss_attention, accuracy_attention = model_attention.evaluate(X_test_cv, y_test)

print(f"\nAttention CNN Test Accuracy: {accuracy_attention}")
print(f"\n Loss Attention: {loss_attention}")

# Predict probabilities for test set
y_pred_prob = model_attention.predict(X_test_cv)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_prob > 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

# AUC Score
auc_score = roc_auc_score(y_test, y_pred_prob)
print(f"AUC Score: {auc_score:.4f}")

# Calculate True Positives, True Negatives, False Positives, False Negatives
tn, fp, fn, tp = cm.ravel()

# Sensitivity (Recall)
sensitivity = tp / (tp + fn)
print(f"Sensitivity (Recall): {sensitivity:.4f}")

# Specificity
specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

# Precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.4f}")

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

# Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_pred)
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

130/130 [==============================] - 2s 13ms/step - loss: 0.0833 - accuracy: 0.9887

Attention CNN Test Accuracy: 0.9887019395828247

 Loss Attention: 0.08331841975450516
130/130 [==============================] - 1s 9ms/step
Confusion Matrix:
[[2103   29]
 [  18 2010]]
AUC Score: 0.9980
Sensitivity (Recall): 0.9911
Specificity: 0.9864
Precision: 0.9858
F1 Score: 0.9884
Matthews Correlation Coefficient (MCC): 0.9774
